In [ ]:
# API Documentation:
# https://bmrs.elexon.co.uk/api-documentation/endpoint/forecast/
# https://api.rebase.energy/weather/docs/v2/#general-usage
# https://api.rebase.energy/challenges/redoc#tag/

In [ ]:
#Imports
import comp_utils
import pandas as pd

# Set up api key
rebase_api_client = comp_utils.RebaseAPI(api_key = open("data/DanMaLeo_key.txt").read())

### RebaseAPI Functions Overview

In [7]:
# National forecast for solar and wind production
# timestamp_utc or settlement_date and settlement_period (23:00 (period 1)- 22:30 (period 48)) are valid time here
solar_wind_forecast = comp_utils.RebaseAPI.get_solar_wind_forecast(rebase_api_client, "2023-10-11")
solar_wind_forecast.tail()

,timestamp_utc,settlement_date,settlement_period,solar_mw,wind_offshore_mw,wind_onshore_mw
43,2023-10-11T19:30:00Z,2023-10-11,42,0.0,5119.954667,4578.129000
44,2023-10-11T20:00:00Z,2023-10-11,43,0.0,5010.592333,4013.771000
45,2023-10-11T20:30:00Z,2023-10-11,44,0.0,5010.592333,4013.771000
46,2023-10-11T21:00:00Z,2023-10-11,45,0.0,4603.203000,3513.663667
47,2023-10-11T21:30:00Z,2023-10-11,46,0.0,4603.203000,3513.663667


In [ ]:
# Forecast for day-ahead-demand
# national_demand: NDF takes transmission losses into account, but not transformer load, pumped storage demand or interconnector demand; data is only reported at national level
# transmition_system_demand: TSDF takes into account transmission losses, transformer load, pumped storage demand and interconnector demand. The data is reported both at national level and at the level of the border areas (system zones)
# boundary: geographical reference area  (https://www.arcgis.com/apps/webappviewer/index.html?id=cb3474a78df24139b1651908ff8c8975)
# no historic data available
da_demand_forecast = comp_utils.RebaseAPI.get_day_ahead_demand_forecast(rebase_api_client) # GReturns list
da_demand_forecast = pd.DataFrame(da_demand_forecast)
da_demand_forecast

<Response [200]>


,timestamp_utc,settlement_date,settlement_period,boundary,publish_time_utc,transmission_system_demand,national_demand
0,2024-10-25T10:00:00Z,2024-10-25,23,N,2024-10-25T09:46:00Z,32789.0,32262.0
1,2024-10-25T10:30:00Z,2024-10-25,24,N,2024-10-25T09:46:00Z,32363.0,31775.0
2,2024-10-25T11:00:00Z,2024-10-25,25,N,2024-10-25T09:46:00Z,32234.0,31730.0
3,2024-10-25T11:30:00Z,2024-10-25,26,N,2024-10-25T09:46:00Z,31982.0,31464.0
4,2024-10-25T12:00:00Z,2024-10-25,27,N,2024-10-25T09:46:00Z,31849.0,31290.0
5,2024-10-25T12:30:00Z,2024-10-25,28,N,2024-10-25T09:46:00Z,31645.0,31110.0
6,2024-10-25T13:00:00Z,2024-10-25,29,N,2024-10-25T09:46:00Z,31619.0,30923.0
7,2024-10-25T13:30:00Z,2024-10-25,30,N,2024-10-25T09:46:00Z,31558.0,30845.0
8,2024-10-25T14:00:00Z,2024-10-25,31,N,2024-10-25T09:46:00Z,31484.0,30800.0
9,2024-10-25T14:30:00Z,2024-10-25,32,N,2024-10-25T09:46:00Z,31611.0,30891.0


In [ ]:
# Forecast for the difference between demand and available generation capacity 2 to 14 days in advance (useful for price forecasting)
# no historical data available
# forecast_date: valid_time
margin_forecast = comp_utils.RebaseAPI.get_margin_forecast(rebase_api_client) # Returns list
margin_forecast = pd.DataFrame(margin_forecast)
margin_forecast

<Response [200]>


,forecast_date,publish_time_utc,margin
0,2024-10-27,2024-10-25T10:00:00Z,12347.0
1,2024-10-28,2024-10-25T10:00:00Z,5932.0
2,2024-10-29,2024-10-25T10:00:00Z,5518.0
3,2024-10-30,2024-10-25T10:00:00Z,6825.0
4,2024-10-31,2024-10-25T10:00:00Z,9441.0
5,2024-11-01,2024-10-25T10:00:00Z,8815.0
6,2024-11-02,2024-10-25T10:00:00Z,11668.0
7,2024-11-03,2024-10-25T10:00:00Z,10783.0
8,2024-11-04,2024-10-25T10:00:00Z,5370.0
9,2024-11-05,2024-10-25T10:00:00Z,7182.0


In [ ]:
# Current weather forecasts for Hornsea 1 from DWD
hornsea_dwd = comp_utils.RebaseAPI.get_hornsea_dwd(rebase_api_client)
hornsea_dwd.tail()

200


,RelativeHumidity_latitude_53.77_longitude_1.702,RelativeHumidity_latitude_53.77_longitude_1.767,RelativeHumidity_latitude_53.77_longitude_1.832,RelativeHumidity_latitude_53.77_longitude_1.897,RelativeHumidity_latitude_53.77_longitude_1.962,RelativeHumidity_latitude_53.77_longitude_2.027,RelativeHumidity_latitude_53.84_longitude_1.702,RelativeHumidity_latitude_53.84_longitude_1.767,RelativeHumidity_latitude_53.84_longitude_1.832,RelativeHumidity_latitude_53.84_longitude_1.897,...,WindSpeed_latitude_54.03_longitude_1.962,WindSpeed_latitude_54.03_longitude_2.027,WindSpeed_latitude_54.1_longitude_1.702,WindSpeed_latitude_54.1_longitude_1.767,WindSpeed_latitude_54.1_longitude_1.832,WindSpeed_latitude_54.1_longitude_1.897,WindSpeed_latitude_54.1_longitude_1.962,WindSpeed_latitude_54.1_longitude_2.027,ref_datetime,valid_datetime
88,90.082863,89.925987,89.925987,89.612244,89.455368,89.141617,90.082863,89.925987,89.925987,89.769119,...,7.761776,7.953669,7.610463,7.590373,7.580990,7.663693,7.753379,7.751348,"Tue, 15 Oct 2024 00:00:00 GMT","Sat, 19 Oct 2024 12:00:00 GMT"
89,93.063484,93.063484,92.749733,92.749733,92.749733,92.592857,93.063484,93.063484,93.063484,92.749733,...,6.717721,7.060050,6.002080,6.228051,6.365960,6.509157,6.646407,6.778482,"Tue, 15 Oct 2024 00:00:00 GMT","Sat, 19 Oct 2024 15:00:00 GMT"
90,90.239738,88.984741,89.769119,91.965363,93.690979,93.690979,90.710358,90.239738,90.710358,92.435982,...,5.106552,5.095594,4.210933,4.495193,4.799330,5.020159,5.171030,5.182777,"Tue, 15 Oct 2024 00:00:00 GMT","Sat, 19 Oct 2024 18:00:00 GMT"
91,84.905998,84.905998,85.062874,85.219749,85.376625,85.376625,86.160995,86.160995,86.160995,86.474747,...,2.594670,2.548558,3.215575,3.039397,2.861455,2.699129,2.587278,2.517273,"Tue, 15 Oct 2024 00:00:00 GMT","Sat, 19 Oct 2024 21:00:00 GMT"
92,84.905998,84.905998,84.905998,84.749123,84.435379,84.278503,86.004120,85.690376,85.376625,84.905998,...,4.179868,4.117596,3.943560,3.937463,3.898464,3.796738,3.730033,3.640726,"Tue, 15 Oct 2024 00:00:00 GMT","Sun, 20 Oct 2024 00:00:00 GMT"


In [22]:
# Aktuelle Wetterforecasts für Hornsea 1 von gfs (ncep)
hornsea_gfs = comp_utils.RebaseAPI.get_hornsea_gfs(rebase_api_client)
hornsea_gfs.tail()

200


,RelativeHumidity_latitude_53.59_longitude_1.522,RelativeHumidity_latitude_53.59_longitude_1.772,RelativeHumidity_latitude_53.59_longitude_2.022,RelativeHumidity_latitude_53.84_longitude_1.522,RelativeHumidity_latitude_53.84_longitude_1.772,RelativeHumidity_latitude_53.84_longitude_2.022,RelativeHumidity_latitude_54.09_longitude_1.522,RelativeHumidity_latitude_54.09_longitude_1.772,RelativeHumidity_latitude_54.09_longitude_2.022,Temperature_latitude_53.59_longitude_1.522,...,WindSpeed_latitude_53.59_longitude_1.772,WindSpeed_latitude_53.59_longitude_2.022,WindSpeed_latitude_53.84_longitude_1.522,WindSpeed_latitude_53.84_longitude_1.772,WindSpeed_latitude_53.84_longitude_2.022,WindSpeed_latitude_54.09_longitude_1.522,WindSpeed_latitude_54.09_longitude_1.772,WindSpeed_latitude_54.09_longitude_2.022,ref_datetime,valid_datetime
204,76.897766,76.495827,76.008087,76.495827,76.197762,75.592598,77.100998,76.098404,75.195175,11.349487,...,9.881656,9.979931,9.432333,9.432333,9.493133,8.456732,8.847167,9.186084,"Mon, 14 Oct 2024 18:00:00 GMT","Wed, 30 Oct 2024 06:00:00 GMT"
205,74.382263,74.599037,73.298386,74.707428,74.599037,73.912582,74.599037,74.310005,74.815819,11.598450,...,8.691893,8.862617,7.987848,8.141713,8.478475,7.494019,7.611781,8.069428,"Mon, 14 Oct 2024 18:00:00 GMT","Wed, 30 Oct 2024 09:00:00 GMT"
206,73.587418,71.528046,70.010620,76.798409,74.310005,73.515160,79.295853,78.202942,76.495827,11.888916,...,8.270662,8.362968,7.372503,7.785734,8.117266,7.770119,7.663439,7.976241,"Mon, 14 Oct 2024 18:00:00 GMT","Wed, 30 Oct 2024 12:00:00 GMT"
207,79.494560,77.701645,75.592598,77.701645,75.195175,73.189995,80.199081,77.001640,75.104851,11.722931,...,8.992424,8.586954,8.083945,8.398210,8.605528,7.767087,7.581845,7.842962,"Mon, 14 Oct 2024 18:00:00 GMT","Wed, 30 Oct 2024 15:00:00 GMT"
208,79.982307,80.596504,80.704895,79.801659,81.102318,81.788773,82.583618,81.391350,81.210701,11.639954,...,8.994325,9.159339,8.712865,9.083673,9.166780,7.993636,8.230933,8.615343,"Mon, 14 Oct 2024 18:00:00 GMT","Wed, 30 Oct 2024 18:00:00 GMT"


In [ ]:
# Current weather forecasts for pes10
pes10_nwp = comp_utils.RebaseAPI.get_pes10_nwp(rebase_api_client,
                                               model="NCEP_GFS")
pes10_nwp

200


,valid_datetime,ref_datetime,SolarDownwardRadiation,CloudCover,Temperature,point,latitude,longitude
0,2024-10-14T18:00:00Z,2024-10-14T18:00:00Z,0.00,0.72,10.52,0,52.487256,0.401245
1,2024-10-14T19:00:00Z,2024-10-14T18:00:00Z,0.00,0.74,10.23,0,52.487256,0.401245
2,2024-10-14T20:00:00Z,2024-10-14T18:00:00Z,0.00,0.49,9.98,0,52.487256,0.401245
3,2024-10-14T21:00:00Z,2024-10-14T18:00:00Z,0.00,0.36,8.92,0,52.487256,0.401245
4,2024-10-14T22:00:00Z,2024-10-14T18:00:00Z,0.00,0.05,7.88,0,52.487256,0.401245
...,...,...,...,...,...,...,...,...
204,2024-10-30T06:00:00Z,2024-10-14T18:00:00Z,0.00,0.92,10.19,19,52.078072,-0.011349
205,2024-10-30T09:00:00Z,2024-10-14T18:00:00Z,115.30,1.00,10.44,19,52.078072,-0.011349
206,2024-10-30T12:00:00Z,2024-10-14T18:00:00Z,95.19,0.91,10.98,19,52.078072,-0.011349
207,2024-10-30T15:00:00Z,2024-10-14T18:00:00Z,32.87,1.00,10.31,19,52.078072,-0.011349


In [ ]:
# Current weather forecasts for the population
demand_nwp = comp_utils.RebaseAPI.get_demand_nwp(rebase_api_client,
                                                 model="NCEP_GFS")
demand_nwp

200


,valid_datetime,ref_datetime,Temperature,WindSpeed,WindDirection,TotalPrecipitation,RelativeHumidity,point,latitude,longitude
0,2024-10-14T18:00:00Z,2024-10-14T18:00:00Z,10.81,2.64,88.22,0.10,94.22,0,51.479,-0.451
1,2024-10-14T19:00:00Z,2024-10-14T18:00:00Z,10.81,2.67,88.24,0.03,94.22,0,51.479,-0.451
2,2024-10-14T20:00:00Z,2024-10-14T18:00:00Z,10.89,2.69,87.59,0.00,94.22,0,51.479,-0.451
3,2024-10-14T21:00:00Z,2024-10-14T18:00:00Z,10.81,2.87,87.74,0.00,93.93,0,51.479,-0.451
4,2024-10-14T22:00:00Z,2024-10-14T18:00:00Z,10.89,3.22,85.67,0.00,93.64,0,51.479,-0.451
...,...,...,...,...,...,...,...,...,...,...
204,2024-10-30T06:00:00Z,2024-10-14T18:00:00Z,9.36,3.99,20.83,0.00,93.21,6,54.297,-1.533
205,2024-10-30T09:00:00Z,2024-10-14T18:00:00Z,10.10,3.29,45.32,0.03,86.56,6,54.297,-1.533
206,2024-10-30T12:00:00Z,2024-10-14T18:00:00Z,11.23,4.04,65.90,0.07,74.71,6,54.297,-1.533
207,2024-10-30T15:00:00Z,2024-10-14T18:00:00Z,10.89,3.88,51.00,0.03,76.30,6,54.297,-1.533


In [ ]:
# Current weather forecasts for any points
# forecast_date: valid_time
# no historical data available
weather_latest = comp_utils.RebaseAPI.query_weather_latest(rebase_api_client, 
                                                           model="DWD_ICON-EU",
                                                           lats='60.1, 61.2, 59.33',
                                                           lons='17.2, 13.1, 18.05',
                                                           variables='Temperature, WindSpeed, CloudCover',
                                                           query_type="GET") # query_type: GET, POST, ...
weather_latest = pd.DataFrame(margin_forecast)
weather_latest

400


,forecast_date,publish_time_utc,margin
0,2024-10-17,2024-10-15T06:00:00Z,7561.0
1,2024-10-18,2024-10-15T06:00:00Z,22547.0
2,2024-10-19,2024-10-15T06:00:00Z,19946.0
3,2024-10-20,2024-10-15T06:00:00Z,26082.0
4,2024-10-21,2024-10-15T06:00:00Z,19064.0
5,2024-10-22,2024-10-15T06:00:00Z,11671.0
6,2024-10-23,2024-10-15T06:00:00Z,11056.0
7,2024-10-24,2024-10-15T06:00:00Z,13598.0
8,2024-10-25,2024-10-15T06:00:00Z,15566.0
9,2024-10-26,2024-10-15T06:00:00Z,16898.0


In [ ]:
# Converts a dataframe to a dataset
latest_dwd_Hornsea1 = comp_utils.weather_df_to_xr(rebase_api_client.get_hornsea_dwd())
latest_dwd_Hornsea1

200


<xarray.Dataset> Size: 162kB
Dimensions:            (ref_datetime: 1, valid_datetime: 93, longitude: 6,
                        latitude: 6)
Coordinates:
  * ref_datetime       (ref_datetime) object 8B 1728820800000000000
  * valid_datetime     (valid_datetime) object 744B 1728820800000000000 ... 1...
  * longitude          (longitude) object 48B '1.702' '1.767' ... '2.027'
  * latitude           (latitude) object 48B '53.77' '53.84' ... '54.03' '54.1'
Data variables:
    RelativeHumidity   (ref_datetime, valid_datetime, longitude, latitude) float64 27kB ...
    Temperature        (ref_datetime, valid_datetime, longitude, latitude) float64 27kB ...
    WindDirection      (ref_datetime, valid_datetime, longitude, latitude) float64 27kB ...
    WindDirection:100  (ref_datetime, valid_datetime, longitude, latitude) float64 27kB ...
    WindSpeed          (ref_datetime, valid_datetime, longitude, latitude) float64 27kB ...
    WindSpeed:100      (ref_datetime, valid_datetime, longitude, latitude) float64 27kB ...